In [ ]:
import rebound
import numpy as np
from numpy.linalg import eig
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
from celmech.secular import LaplaceLagrangeSystem
from celmech.poincare import Poincare
from celmech.disturbing_function import laplace_b as b
from scipy.optimize import fsolve
import cmath

# Mapping eigenmodes and eccentricity vectors

In [ ]:
def calcChi(mass,alpha):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    
    alpha12, alpha23 = alpha
    alpha13 = alpha12*alpha23
    ec12 = alpha12**(-1/4)*alpha23**(3/4)*alpha23**(-1/8)*(1-alpha12)
    ec23 = alpha23**(-1/2)*alpha12**(1/8)*(1-alpha23)
    ec13 = alpha13**(-1/2)*(1-alpha13)
    eta = (ec12 - ec23)/ec13
    
    chi12, chi23 = (1-eta)**3*(3+eta)*mu1, (1+eta)**3*(3-eta)*mu3
    return chi12/(chi12+chi23), chi23/(chi12+chi23)

In [ ]:
def getT(mass, alpha):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    
    chi12, chi23 = calcChi(mass,alpha)
    T = np.array([[chi12, -1, chi23],
                 [-1, 0, 1],
                 [mu1, mu2, mu3]])
    return T

In [ ]:
def AtoE(alpha, mass, amp, phi):
    A1, A2, A3 = amp
    p1, p2, p3 = phi
    
    invT = np.linalg.inv(getT(mass, alpha))
    A_vec = np.array([[A1*cmath.exp(1j*p1)], [A2*cmath.exp(1j*p2)], [A3*cmath.exp(1j*p3)]])
    e_vec = invT @ A_vec
    
    e1, e2, e3 = np.abs(e_vec[0][0]), np.abs(e_vec[1][0]), np.abs(e_vec[2][0])
    pom1, pom2, pom3 = cmath.phase(e_vec[0][0]), cmath.phase(e_vec[1][0]), cmath.phase(e_vec[2][0])
    
    return (e1, e2, e3), (pom1, pom2, pom3)

def EtoA(alpha, mass, ecc, pomega):
    e1, e2, e3 = ecc
    pom1, pom2, pom3 = pomega
    
    T = getT(mass, alpha)
    e_vec = np.array([[e1*cmath.exp(1j*pom1)], [e2*cmath.exp(1j*pom2)], [e3*cmath.exp(1j*pom3)]])
    A_vec = T @ e_vec
    
    amp1, amp2, amp3 = np.abs(A_vec[0][0]), np.abs(A_vec[1][0]), np.abs(A_vec[2][0])
    phi1, phi2, phi3 = cmath.phase(A_vec[0][0]), cmath.phase(A_vec[1][0]), cmath.phase(A_vec[2][0])
    
    return amp1, amp2, amp3, phi1, phi2, phi3

# Make and run simulation

In [ ]:
def make_sim(alpha, mass, ecc, pom, Lambda):
    mu1, mu2, mu3 = mass
    alpha12, alpha23 = alpha
    P1, P2, P3 = alpha12**(3/2), 1, (alpha23)**(-3/2)
    ecc1, ecc2, ecc3 = ecc
    pomega1, pomega2, pomega3 = pom
    l1, l2, l3 = Lambda
    
    # start simulation
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')

    # add star, planet 1, planet 2
    sim.add(m=1.)
    sim.add(m=mu1, P=P1, e=ecc1, pomega=pomega1, l=l1)
    sim.add(m=mu2, P=P2, e=ecc2, pomega=pomega2, l=l2)
    sim.add(m=mu3, P=P3, e=ecc3, pomega=pomega3, l=l3)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.ri_whfast.safe_mode = 0
    sim.dt = sim.particles[1].P/12
    sim.collision = "direct"
    
    return sim

In [ ]:
def run(sim):
    ps = sim.particles
    
    ec13 = 1-ps[1].a/ps[3].a
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    m_tot = m1 + m2 + m3
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    Tsec = 2*np.pi/(1/2*m_tot/ps[0].m/ec13**2)*ps[1].P
#     print(Tsec)
    
    chi12, chi23 = calcChi((m1,m2,m3),(ps[1].a/ps[2].a, ps[2].a/ps[3].a))
    
    Nout = 1000
    times = np.linspace(0,2*Tsec, Nout)
    e1, e2, e3, e12, e23, question = np.zeros(Nout), np.zeros(Nout), np.zeros(Nout), np.zeros(Nout), np.zeros(Nout), np.zeros(Nout)
    for i, time in enumerate(times):
        sim.integrate(time)    
        e1x, e2x, e3x = [p.e*np.cos(p.pomega) for p in ps[1:]]
        e1y, e2y, e3y = [p.e*np.sin(p.pomega) for p in ps[1:]]
        
        S1ax, S1ay = chi23*(e3x-e2x)-chi12*(e2x-e1x), chi23*(e3y-e2y)-chi12*(e2y-e1y)       
        S2ax, S2ay = e3x-e1x, e3y-e1y
        S3ax, S3ay = (mu1*e1x+mu2*e2x+mu3*e3x), (mu1*e1y+mu2*e2y+mu3*e3y)
        
        e1[i] = np.sqrt(e1x**2+e1y**2)
        e2[i] = np.sqrt(e2x**2+e2y**2)
        e3[i] = np.sqrt(e3x**2+e3y**2)
        
        e12[i] = np.sqrt((e2x-e1x)**2+(e2y-e1y)**2)
        e23[i] = np.sqrt((e3x-e2x)**2+(e3y-e2y)**2)
        fac = 1/2*((chi23-chi12)-(mu3-mu1))
        question[i] = np.sqrt((S3ax-S1ax+fac*S2ax)**2+(S3ay-S1ay+fac*S2ay)**2)
     
    return times, e1, e2, e3, e12, e23, question